In [1]:
import pandas as pd
import numpy as np
from clases.NeuronalNetwork import SimpleNeuralNetwork
from clases.Optimizer import SGDOptimizer, AdamOptimizer
from clases.Trainer import Trainer
from clases.LossFunction import *
import warnings
from clases.ActivationFunction import *

In [2]:
warnings.filterwarnings("ignore")

data = pd.read_csv('../datos/mnist_data.csv')
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

test_size = int(0.1 * m)
data_test = data[:test_size, :].T  # El 10% inicial para el conjunto de prueba

# El conjunto provisional restante (90%)
remaining_data = data[test_size:, :]

train_size = int(0.8 * m)
data_train = remaining_data[:train_size, :].T  # El siguiente 80% para entrenamiento
data_validation = remaining_data[train_size:, :].T  # El resto 10% para validación

X_train, Y_train = data_train[1:], data_train[0]
X_val, Y_val = data_validation[1:], data_validation[0]
X_test, Y_test = data_test[1:], data_test[0]

In [3]:
model = SimpleNeuralNetwork(input_size=784, output_size=10, activation_function=relu, activation_function_d=relu_derivative, output_function=softmax)
optimizer = SGDOptimizer(learning_rate=0.001)
trainer = Trainer(model, optimizer, cross_entropy_loss)
trainer.train(X_train, Y_train,X_val,Y_val, epochs=100)

--------------------------------------------------------------------
| Epoch   0 | time:  0.14s | val loss 2.338 | valid accuracy 0.108 |
--------------------------------------------------------------------
| Epoch  10 | time:  0.14s | val loss 2.086 | valid accuracy 0.245 |
--------------------------------------------------------------------
| Epoch  20 | time:  0.13s | val loss 1.700 | valid accuracy 0.418 |
--------------------------------------------------------------------
| Epoch  30 | time:  0.14s | val loss 1.293 | valid accuracy 0.593 |
--------------------------------------------------------------------
| Epoch  40 | time:  0.13s | val loss 0.996 | valid accuracy 0.704 |
--------------------------------------------------------------------
| Epoch  50 | time:  0.13s | val loss 0.820 | valid accuracy 0.759 |
--------------------------------------------------------------------
| Epoch  60 | time:  0.14s | val loss 0.712 | valid accuracy 0.787 |
----------------------------------

In [4]:
model = SimpleNeuralNetwork(input_size=784, output_size=10, activation_function=relu, activation_function_d=relu_derivative, output_function=softmax)
optimizer = AdamOptimizer(learning_rate=0.001)
trainer = Trainer(model, optimizer, cross_entropy_loss)
trainer.train(X_train, Y_train,X_val,Y_val, epochs=100)

--------------------------------------------------------------------
| Epoch   0 | time:  0.17s | val loss 2.214 | valid accuracy 0.184 |
--------------------------------------------------------------------
| Epoch  10 | time:  0.16s | val loss 0.990 | valid accuracy 0.678 |
--------------------------------------------------------------------
| Epoch  20 | time:  0.14s | val loss 0.555 | valid accuracy 0.827 |
--------------------------------------------------------------------
| Epoch  30 | time:  0.14s | val loss 0.441 | valid accuracy 0.867 |
--------------------------------------------------------------------
| Epoch  40 | time:  0.16s | val loss 0.380 | valid accuracy 0.889 |
--------------------------------------------------------------------
| Epoch  50 | time:  0.13s | val loss 0.348 | valid accuracy 0.900 |
--------------------------------------------------------------------
| Epoch  60 | time:  0.15s | val loss 0.327 | valid accuracy 0.909 |
----------------------------------